# Proyecto Final - Segunda Entrega

In [1]:
# Librerías y módulos
# General
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit Learn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
# Estilo de los gráficos
sns.set()

# Ignorar warnings
import warnings
warnings.filterwarnings('ignore')

#### En primera instancia, se realiza el preprocesamiento del dataset hastla llegar a la definición de la matriz X y el vector y sobre el cual se realizan las predicciones.

#### A grandesr rasgos, serán eliminados los outliers, las filas que contuvieran valores nulos y la información será normalizada con el fin de obtener un dataset correctamente balanceado.

In [2]:
# Cargo el dataset
df_1 = pd.read_csv('startup data.csv')
df_1.head()

,Unnamed: 0,state_code,latitude,longitude,zip_code,id,city,Unnamed: 6,name,labels,...,object_id,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,is_top500,status
0,1005,CA,42.358880,-71.056820,92101,c:6669,San Diego,NaN,Bandsintown,1,...,c:6669,0,1,0,0,0,0,1.0000,0,acquired
1,204,CA,37.238916,-121.973718,95032,c:16283,Los Gatos,NaN,TriCipher,1,...,c:16283,1,0,0,1,1,1,4.7500,1,acquired
2,1001,CA,32.901049,-117.192656,92121,c:65620,San Diego,San Diego CA 92121,Plixi,1,...,c:65620,0,0,1,0,0,0,4.0000,1,acquired
3,738,CA,37.320309,-122.050040,95014,c:42668,Cupertino,Cupertino CA 95014,Solidcore Systems,1,...,c:42668,0,0,0,1,1,1,3.3333,1,acquired
4,1002,CA,37.779281,-122.419236,94105,c:65806,San Francisco,San Francisco CA 94105,Inhale Digital,0,...,c:65806,1,1,0,0,0,0,1.0000,1,closed


In [3]:
df_1.shape

(923, 49)

#### Se consideran como datos erroneos las filas que contengan edades negativas y por lo tanto se las elimina del dataset.

In [4]:
df_1 = df_1.drop(df_1[
(df_1['age_first_funding_year']<0) |
(df_1['age_last_funding_year']<0) |
(df_1['age_first_milestone_year']<0) |
(df_1['age_last_milestone_year']<0)
].index)

df_1.head()

,Unnamed: 0,state_code,latitude,longitude,zip_code,id,city,Unnamed: 6,name,labels,...,object_id,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,is_top500,status
0,1005,CA,42.358880,-71.056820,92101,c:6669,San Diego,NaN,Bandsintown,1,...,c:6669,0,1,0,0,0,0,1.0000,0,acquired
1,204,CA,37.238916,-121.973718,95032,c:16283,Los Gatos,NaN,TriCipher,1,...,c:16283,1,0,0,1,1,1,4.7500,1,acquired
2,1001,CA,32.901049,-117.192656,92121,c:65620,San Diego,San Diego CA 92121,Plixi,1,...,c:65620,0,0,1,0,0,0,4.0000,1,acquired
3,738,CA,37.320309,-122.050040,95014,c:42668,Cupertino,Cupertino CA 95014,Solidcore Systems,1,...,c:42668,0,0,0,1,1,1,3.3333,1,acquired
4,1002,CA,37.779281,-122.419236,94105,c:65806,San Francisco,San Francisco CA 94105,Inhale Digital,0,...,c:65806,1,1,0,0,0,0,1.0000,1,closed


In [5]:
df_1.shape

(840, 49)

#### Además, se eliminan las columnas que no aportan a la construcción del modelo de predicción. Como comentario, la columna "labels" es igual a la columna "status" pero de caracter numérico y es al mismo tiempo la variable a predecir.

In [6]:
df_1.columns

Index(['Unnamed: 0', 'state_code', 'latitude', 'longitude', 'zip_code', 'id',
       'city', 'Unnamed: 6', 'name', 'labels', 'founded_at', 'closed_at',
       'first_funding_at', 'last_funding_at', 'age_first_funding_year',
       'age_last_funding_year', 'age_first_milestone_year',
       'age_last_milestone_year', 'relationships', 'funding_rounds',
       'funding_total_usd', 'milestones', 'state_code.1', 'is_CA', 'is_NY',
       'is_MA', 'is_TX', 'is_otherstate', 'category_code', 'is_software',
       'is_web', 'is_mobile', 'is_enterprise', 'is_advertising',
       'is_gamesvideo', 'is_ecommerce', 'is_biotech', 'is_consulting',
       'is_othercategory', 'object_id', 'has_VC', 'has_angel', 'has_roundA',
       'has_roundB', 'has_roundC', 'has_roundD', 'avg_participants',
       'is_top500', 'status'],
      dtype='object')

In [7]:
df_1 = df_1.drop(columns=['Unnamed: 0', 
                          'state_code',
                          'latitude',
                          'longitude',
                          'zip_code',
                          'id',
                          'city',
                          'Unnamed: 6',
                          'name',
                          'founded_at',
                          'closed_at',
                          'first_funding_at',
                          'last_funding_at',
                          'state_code.1',
                          'category_code',
                          'object_id',
                          'status'
                                        ])

In [8]:
df_1.shape

(840, 32)

In [9]:
df_1.isna().sum()

labels                        0
age_first_funding_year        0
age_last_funding_year         0
age_first_milestone_year    144
age_last_milestone_year     144
relationships                 0
funding_rounds                0
funding_total_usd             0
milestones                    0
is_CA                         0
is_NY                         0
is_MA                         0
is_TX                         0
is_otherstate                 0
is_software                   0
is_web                        0
is_mobile                     0
is_enterprise                 0
is_advertising                0
is_gamesvideo                 0
is_ecommerce                  0
is_biotech                    0
is_consulting                 0
is_othercategory              0
has_VC                        0
has_angel                     0
has_roundA                    0
has_roundB                    0
has_roundC                    0
has_roundD                    0
avg_participants              0
is_top50

#### Valores nulos en las columnas "age_first_milestone_year" y "age_last_milestone_year" indican que nunca tuvieron un "milestone". Se decide rellenar dichos valores nulos con ceros.

In [10]:
df_1['age_first_milestone_year'] = df_1['age_first_milestone_year'].fillna(0)
df_1['age_last_milestone_year'] = df_1['age_last_milestone_year'].fillna(0)

df_1.isna().sum()

labels                      0
age_first_funding_year      0
age_last_funding_year       0
age_first_milestone_year    0
age_last_milestone_year     0
relationships               0
funding_rounds              0
funding_total_usd           0
milestones                  0
is_CA                       0
is_NY                       0
is_MA                       0
is_TX                       0
is_otherstate               0
is_software                 0
is_web                      0
is_mobile                   0
is_enterprise               0
is_advertising              0
is_gamesvideo               0
is_ecommerce                0
is_biotech                  0
is_consulting               0
is_othercategory            0
has_VC                      0
has_angel                   0
has_roundA                  0
has_roundB                  0
has_roundC                  0
has_roundD                  0
avg_participants            0
is_top500                   0
dtype: int64

In [11]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 840 entries, 0 to 922
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   labels                    840 non-null    int64  
 1   age_first_funding_year    840 non-null    float64
 2   age_last_funding_year     840 non-null    float64
 3   age_first_milestone_year  840 non-null    float64
 4   age_last_milestone_year   840 non-null    float64
 5   relationships             840 non-null    int64  
 6   funding_rounds            840 non-null    int64  
 7   funding_total_usd         840 non-null    int64  
 8   milestones                840 non-null    int64  
 9   is_CA                     840 non-null    int64  
 10  is_NY                     840 non-null    int64  
 11  is_MA                     840 non-null    int64  
 12  is_TX                     840 non-null    int64  
 13  is_otherstate             840 non-null    int64  
 14  is_softwar

#### También, y con el objetivo de reducir posibles sezgos, se recomienda normalizar la información previamente. Esta normalización se va aplicar unicamente sobre las variables puramente numéricas y no sobre las que surgieron del porceso de one hot encoding.

In [12]:
normalizar_columnas= [
    'age_first_funding_year',
    'age_last_funding_year',
    'age_first_milestone_year',
    'age_last_milestone_year',
    'relationships',          
    'funding_rounds',              
    'funding_total_usd',           
    'milestones',
    'avg_participants'
    ]

In [13]:
for i in normalizar_columnas:
    df_1[i] = (df_1[i] - df_1[i].mean()) / df_1[i].std() 

df_1.head()

,labels,age_first_funding_year,age_last_funding_year,age_first_milestone_year,age_last_milestone_year,relationships,funding_rounds,funding_total_usd,milestones,is_CA,...,is_consulting,is_othercategory,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,is_top500
0,1,-0.069786,-0.383699,0.654706,0.756949,-0.652793,0.484014,-0.134415,0.889515,1,...,0,1,0,1,0,0,0,0,-0.994238,0
1,1,1.083016,1.997422,1.475847,0.845494,0.171375,1.196048,0.065524,-0.618442,1,...,0,0,1,0,0,1,1,1,0.971545,1
2,1,-0.557243,-1.054264,-0.473528,-0.564642,-0.378071,-0.940055,-0.123217,0.135537,1,...,0,0,0,0,1,0,0,0,0.578389,1
3,1,0.283745,0.403494,1.123498,0.550893,-0.378071,0.484014,0.065021,-0.618442,1,...,0,0,0,0,0,1,1,1,0.228899,1
4,0,-0.971164,-0.837891,-0.972150,-1.201289,-0.790155,-0.228020,-0.129760,-0.618442,1,...,0,0,1,1,0,0,0,0,-0.994238,1


#### Deben definirse las variables que formarán la matriz X. En este caso, todas las columnas excepto aquella que hace referencia al "status" o "labels" de la startup.

In [14]:
X = df_1.drop(columns=['labels'])
X.head()

,age_first_funding_year,age_last_funding_year,age_first_milestone_year,age_last_milestone_year,relationships,funding_rounds,funding_total_usd,milestones,is_CA,is_NY,...,is_consulting,is_othercategory,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,is_top500
0,-0.069786,-0.383699,0.654706,0.756949,-0.652793,0.484014,-0.134415,0.889515,1,0,...,0,1,0,1,0,0,0,0,-0.994238,0
1,1.083016,1.997422,1.475847,0.845494,0.171375,1.196048,0.065524,-0.618442,1,0,...,0,0,1,0,0,1,1,1,0.971545,1
2,-0.557243,-1.054264,-0.473528,-0.564642,-0.378071,-0.940055,-0.123217,0.135537,1,0,...,0,0,0,0,1,0,0,0,0.578389,1
3,0.283745,0.403494,1.123498,0.550893,-0.378071,0.484014,0.065021,-0.618442,1,0,...,0,0,0,0,0,1,1,1,0.228899,1
4,-0.971164,-0.837891,-0.972150,-1.201289,-0.790155,-0.228020,-0.129760,-0.618442,1,0,...,0,0,1,1,0,0,0,0,-0.994238,1


In [15]:
X.shape

(840, 31)

 #### Por otro lado, el vector "y" sobre el cual se predice se encuentra representado por la columna "status" o labels la cual es de caracter binario e indica si una startup fue adquirida (éxito=1) o debio cerrar (fracaso=0). 

In [16]:
df_1['labels'].value_counts()

1    552
0    288
Name: labels, dtype: int64

In [17]:
df_1['labels'].isna().sum()

0

In [18]:
# Defino el vector y:
y = df_1['labels']
print(y.shape, type(y[0]))

(840,) <class 'numpy.int64'>


In [19]:
# Modifico el tipo de la variable y
y = y.astype('int')

#### Ya se encuentran definidos la matriz X y el vector y que era el objetivo de esta primera parte. A continuación, se propone desarrollar un modelo de predicción a partir del algoritmo "Decision Tree Classifier" o arbol de decisión.

In [20]:
# Divido la matriz X y el vector y en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [21]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(588, 31) (252, 31) (588,) (252,)


In [22]:
# Creo el modelo
tree = DecisionTreeClassifier(max_depth=5, random_state=42) # Se propone un valor de max_depth=5

In [23]:
# Entreno el modelo
tree.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, random_state=42)

In [24]:
# Realizo una predicción
y_train_pred_1 = tree.predict(X_train) # Predicción en Train
y_test_pred_1 = tree.predict(X_test) # Predicción en Test

In [25]:
# Calculo el accuracy en Train
train_accuracy_1 = accuracy_score(y_train, y_train_pred_1)

# Calculo el accuracy en Test
test_accuracy_1 = accuracy_score(y_test, y_test_pred_1)

print('% de aciertos sobre el set de entrenamiento:', round(100*train_accuracy_1, 1))
print('% de aciertos sobre el set de evaluación:',round(100*test_accuracy_1, 1))

% de aciertos sobre el set de entrenamiento: 84.4
% de aciertos sobre el set de evaluación: 77.8


#### Otra manera de cuantificar la calidad del modelo es através de la matriz de confusión.
#### En la misma se indican:
    - Verdaderos Positivos
    - Falsos Positivos
    - Verdaderos Negativos
    - Falsos Negativos
#### Tanto Falsos positivos como negativos son indeseados. Para el caso particular de este problema, se considera igualmente más riesgoso o se castiga más a modelos con mayor cantidad de falsos positivos, ya que implica un gran riego de inversión indicar que una startup fue exitosa cuando en realidad no lo fue.

In [26]:
from sklearn.metrics import confusion_matrix

#### Se determina la matriz de confusión a partir de la comparación entre el vector "y_test" y el vector "y_test_pred_1".

In [27]:
CM_1 = confusion_matrix(y_test, y_test_pred_1)
CM_1

array([[ 38,  41],
       [ 15, 158]], dtype=int64)

#### Por otro lado, se propone hacer uso de otro algoritmol llamado Random Forest. El mismo es muy similar al Decision Tree Classifier. Difieren en que Random Forest utiliza "n" árboles predictores y por lo tanto la predicción final se resuelve por votación y no por el resultado que arroja un único árbol.

In [28]:
# Importo las librerías necesarias
from sklearn.ensemble import RandomForestClassifier

In [29]:
# Creo un modelo Random Forest
random_forest = RandomForestClassifier(random_state=42, n_estimators=50, max_depth=5) # Se propone utilizar 50 árboles

# Entreno al modelo
random_forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, n_estimators=50, random_state=42)

In [30]:
# Realizo una predicción
y_train_pred_2 = random_forest.predict(X_train) # Predicción en Train
y_test_pred_2 = random_forest.predict(X_test) # Predicción en Test

In [31]:
# Calculo el accuracy en Train
train_accuracy_2 = accuracy_score(y_train, y_train_pred_2)

# Calculo el accuracy en Test
test_accuracy_2 = accuracy_score(y_test, y_test_pred_2)

print('% de aciertos sobre el set de entrenamiento:', round(100*train_accuracy_2, 1))
print('% de aciertos sobre el set de evaluación:',round(100*test_accuracy_2, 1))

% de aciertos sobre el set de entrenamiento: 85.0
% de aciertos sobre el set de evaluación: 81.0


#### Se calcula nuevamente la matriz de confusión para este nuevo modelo.

In [32]:
CM_2 = confusion_matrix(y_test, y_test_pred_2)
CM_2

array([[ 38,  41],
       [  7, 166]], dtype=int64)

#### En búsqueda de un mejor modelo, se propone hacer un tuneo de los parametro que caracterizan al modelo de random forest.

In [33]:
from sklearn.model_selection import GridSearchCV

In [34]:
random_forest.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 50,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [36]:
# Definicion de los Hyperparámetros
param_grid =  {
    'bootstrap': [True, False],
    'max_depth': [3, 5, 7, 10],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [25, 50, 75, 100, 200]
    }

In [38]:
random_forest_tuned = GridSearchCV(random_forest, param_grid=param_grid, cv=5)

In [39]:
random_forest_tuned.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(max_depth=5, n_estimators=50,
                                              random_state=42),
             param_grid={'bootstrap': [True, False], 'max_depth': [3, 5, 7, 10],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [25, 50, 75, 100, 200]})

In [42]:
# Realizo una predicción
random_forest_tuned.best_params_

{'bootstrap': True,
 'max_depth': 5,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 25}

#### Con estos nuevos mejores parámetros se calculan nuevamente las métricas del nuevo modelo mejorado. 

In [50]:
random_forest_2 = RandomForestClassifier(
                    bootstrap=True,
                    max_features='auto',
                    min_samples_leaf=1,
                    random_state=42,
                    n_estimators=25,
                    max_depth=5,
                    min_samples_split=5
                                        )

In [51]:
random_forest_2.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, min_samples_split=5, n_estimators=25,
                       random_state=42)

In [52]:
# Realizo una predicción
y_train_pred_3 = random_forest_2.predict(X_train) # Predicción en Train
y_test_pred_3 = random_forest_2.predict(X_test) # Predicción en Test

In [53]:
# Calculo el accuracy en Train
train_accuracy_3 = accuracy_score(y_train, y_train_pred_3)

# Calculo el accuracy en Test
test_accuracy_3 = accuracy_score(y_test, y_test_pred_3)

print('% de aciertos sobre el set de entrenamiento:', round(100*train_accuracy_3, 1))
print('% de aciertos sobre el set de evaluación:',round(100*test_accuracy_3, 1))

% de aciertos sobre el set de entrenamiento: 84.4
% de aciertos sobre el set de evaluación: 81.3


In [55]:
CM_3 = confusion_matrix(y_test, y_test_pred_3)
CM_3

array([[ 39,  40],
       [  7, 166]], dtype=int64)

#### En este caso se disminuyo la cantidad de falsos positivos en una unidad solamente. Invita a pensar que el costo computacional invertido no valio la pena

#### **Algunas conclusiones:**
#### - Se propuso realizar predicciones sobre la variable "status" con dos algoritmos similares Decision Tree Classifier y Random Forest.
#### - Aunque no muy significativos, Random Forest arrojo resultados levemente superiores desde una primera instacia. Resulta interesante de notar, que ambos algoritmos devolvieron la misma cantidad de falsos positivos.
#### - Se realizo un hipertuneo de parámetros que buscará maximizar los resultados obtenidos para el algoritmo de random forest. De todos modos, la calidad de los resultados no mejoro significativamente.
#### - Es posible, que con otros algoritmos se puedan obtener resultado más certeros y por lo tanto se seguirá investigando en este sentido en próximas entregas. De todas formas, los resultadoshasta aqui obtenidos no dejan de ser buenos ya que se obtuvieron métricas aceptables.